In [5]:
import numpy as np
import pandas as pd
import requests
import json

In [21]:
data = pd.read_csv('data/DFR_Data_FY2022.csv')
print(data.columns)
data = data[['provfs','dfr_provname','provcity','state','chainnam']]
data.to_csv('data/DFR_Data_FY2022_short.csv')


Index(['provfs', 'dfr_provname', 'provcity', 'state', 'network', 'chainnam',
       'cw_owner_f', 'allmcpatq1_f', 'allmcFcovpatq1_f', 'allmcFcovpatPq1_f',
       ...
       'p_mefavfmy1_f', 'p_mefgraftmy1_f', 'p_mefcathmy1_f',
       'p_mefoumaccessmy1_f', 'p_avfpresentmy1_f', 'p_hemomy1_f', 'p_nmy1_f',
       'p_allmdcrmy2_f', 'p_insmdcdmy2_f', 'p_insempmy2_f'],
      dtype='object', length=1024)


/home/erichschulman/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (95,96,97,98,99,100,101,102,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,553,554,555,556,558,559,560,561,562,563,564,565,566,567,568,569,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
data2 = pd.read_csv('data/ESRD_Provider_PUF.csv')
print(data2.columns)
data2['provfs'] = data2['state_code'].astype('str') + data2['prov_nbr'].astype('str')
data2['provfs'] = data2['provfs'].astype('int')
merge_data = pd.merge(data2,data[['provfs','chainnam']],how='left',left_on='provfs',right_on='provfs')

Index(['Unnamed: 0', 'state_code', 'prov_nbr', 'name', 'address', 'city',
       'state', 'zip'],
      dtype='object')


In [30]:
merge_data['chainnam'] = merge_data['chainnam'].astype('str')
merge_data['davita'] = merge_data['chainnam'].apply(lambda x : 'DAVITA' in x)
merge_data['fresenius'] = merge_data['chainnam'].apply(lambda x : 'FRESENIUS' in x)
grouped_data = merge_data.groupby('state').agg({'provfs':'count','davita':'sum','fresenius':'sum'})
grouped_data.to_csv('data/Provider_Per_State.csv')
print(grouped_data)

       provfs  davita  fresenius
state                           
AK          9       0          7
AL        187      61         93
AR         73      36         12
AS          1       0          0
AZ        133      48         55
CA        721     337        136
CO         85      39         19
CT         53      29         13
DC         23      12          7
DE         35       0         23
FL        510     215        151
GA        380     151        109
GU          5       0          0
HI         33       2         14
IA         71      30         15
ID         27       9         12
IL        353     133        145
IN        181      69         82
KS         67      38         23
KY        128      41         65
LA        193      38        121
MA         92      13         37
MD        173      85         50
ME         20       4         11
MI        229      94         82
MN        124      50         37
MO        173      62         54
MP          2       0          0
MS        

In [ ]:
addresses = ['50 w 96th street, new york, new york, 10025',"61 Deal St, Harrington Park, NJ 07640" ]

base_url = "https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?address=%s&\
benchmark=Public_AR_Census2020&vintage=Census2010_Census2020&format=json"%(addresses[0])
response=requests.get(base_url)
print(json.loads(response.text)['result']['addressMatches'][0]['geographies']['Counties'][0]['GEOID'])